**Create a difference map for MRT**

In [8]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

masker = NiftiMasker()
mrt_gsp1000_mf = masker.transform('GSP1000_MF_networks/MRT_bilateral_T.nii.gz')
mrt_yeo1000 = masker.transform('yeo1000_dil_networks/MRT_bilateral_T.nii.gz')

# Let's threshold all negative values to 0
mrt_gsp1000_mf = mrt_gsp1000_mf.at[mrt_gsp1000_mf < 0].set(0)
mrt_yeo1000 = mrt_yeo1000.at[mrt_yeo1000 < 0].set(0)


mrt_difference_map = mrt_gsp1000_mf - mrt_yeo1000
mrt_difference_map = masker.inverse_transform(mrt_difference_map)
mrt_difference_map.to_filename('brain_maps/mrt_difference_map.nii.gz')


**Now, create a difference map for the VSM as well**

In [9]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

masker = NiftiMasker()
vsm_gsp1000_mf = masker.transform('GSP1000_MF_networks/VSM_bilateral_T.nii.gz')
vsm_yeo1000 = masker.transform('yeo1000_dil_networks/VSM_bilateral_T.nii.gz')

# Let's threshold all negative values to 0
vsm_gsp1000_mf = vsm_gsp1000_mf.at[vsm_gsp1000_mf < 0].set(0)
vsm_yeo1000 = vsm_yeo1000.at[vsm_yeo1000 < 0].set(0)

vsm_difference_map = vsm_gsp1000_mf - vsm_yeo1000
vsm_difference_map = masker.inverse_transform(vsm_difference_map)
vsm_difference_map.to_filename('brain_maps/vsm_difference_map.nii.gz')

**Create a difference map for the 1-r derived brainstem ROI**

In [10]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

masker = NiftiMasker()
data_gsp1000_mf = masker.transform('GSP1000_MF_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi_T.nii.gz')
data_yeo1000 = masker.transform('yeo1000_dil_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi_T.nii.gz')

# Let's threshold all negative values to 0
data_gsp1000_mf = data_gsp1000_mf.at[data_gsp1000_mf < 0].set(0)
data_yeo1000 = data_yeo1000.at[data_yeo1000 < 0].set(0)

data_difference_map = data_gsp1000_mf - data_yeo1000
data_difference_map = masker.inverse_transform(data_difference_map)
data_difference_map.to_filename('brain_maps/brainstem_roi_difference.nii.gz')

**Calculate Signal to Noise Ratio (SNR) for the 1-r derived brainstem ROI**

In [6]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

# Load the brain maps using NiftiMasker
masker = NiftiMasker()

data_gsp1000_mf = masker.transform('GSP1000_MF_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi_T.nii.gz')
data_yeo1000 = masker.transform('yeo1000_dil_networks/1-r_01_gsp1000MF_vs_yeo1000_masked_brainstem_roi_T.nii.gz')

# Threshold all negative values to 0
# data_gsp1000_mf = data_gsp1000_mf.at[data_gsp1000_mf < 0].set(0)
# data_yeo1000 = data_yeo1000.at[data_yeo1000 < 0].set(0)

difference_data = data_gsp1000_mf - data_yeo1000

# Load the difference map
# path = 'brain_maps/brainstem_roi_difference.nii.gz'
# difference_data = masker.transform(path) # positive values are GSP1000_MF > Yeo1000

# Threshold all negative values to 0 in the difference map
# difference_data = difference_data.at[difference_data < 0].set(0)

# Calculate the signal power (mean squared value of the original brain maps)
signal_power_gsp1000_mf = np.mean(data_gsp1000_mf ** 2)
signal_power_yeo1000 = np.mean(data_yeo1000 ** 2)

# Calculate the noise power (mean squared value of the difference map)
noise_power = np.mean(difference_data ** 2)

# Calculate SNR for each connectome
snr_gsp1000_mf = signal_power_gsp1000_mf / noise_power
snr_gsp1000_mf_db = 10 * np.log10(snr_gsp1000_mf)

snr_yeo1000 = signal_power_yeo1000 / noise_power
snr_yeo1000_db = 10 * np.log10(snr_yeo1000)

print(f"SNR for GSP1000_MF: {snr_gsp1000_mf:.2f}")
print(f"SNR for GSP1000_MF (dB): {snr_gsp1000_mf_db:.2f} dB")
print(f"SNR for Yeo1000: {snr_yeo1000:.2f}")
print(f"SNR for Yeo1000 (dB): {snr_yeo1000_db:.2f} dB")

# # Visualize the difference map
# difference_img = masker.inverse_transform(difference_data)
# difference_img.to_filename('difference_map_visualization.nii.gz')

# import matplotlib.pyplot as plt
# slice_index = difference_img.shape[2] // 2
# plt.imshow(difference_img.get_fdata()[:, :, slice_index], cmap='hot')
# plt.title('Difference Map')
# plt.colorbar()
# plt.show()


SNR for GSP1000_MF: 3.03
SNR for GSP1000_MF (dB): 4.81 dB
SNR for Yeo1000: 3.17
SNR for Yeo1000 (dB): 5.02 dB


**Let's correlate ABCD's VSM network with Yeo and GSP1000's**

In [18]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker, correlate_map_with_references
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

abcd_vsm_path = 'abcd_networks/VSM_bilateral_lesion_T.nii.gz'
gsp1000mf_path = 'GSP1000_MF_networks/VSM_bilateral_T.nii.gz'
yeo1000_path = 'yeo1000_dil_networks/VSM_bilateral_T.nii.gz'

correlations = correlate_map_with_references(abcd_vsm_path, [gsp1000mf_path, yeo1000_path], mask='MNI152_T1_2mm_brain_mask_dil')
display(correlations)

correlations = correlate_map_with_references(gsp1000mf_path, yeo1000_path, mask='MNI152_T1_2mm_brain_mask_dil')
correlations


array([0.26287946, 0.21348274], dtype=float32)

array([0.8513432], dtype=float32)

**Let's correlate ABCD's MRT network with Yeo and GSP1000's**

In [19]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker, correlate_map_with_references
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

abcd_mrt_path = 'abcd_networks/MRT_bilateral_lesion_T.nii.gz'
gsp1000mf_path = 'GSP1000_MF_networks/MRT_bilateral_T.nii.gz'
yeo1000_path = 'yeo1000_dil_networks/MRT_bilateral_T.nii.gz'

correlations = correlate_map_with_references(abcd_mrt_path, [gsp1000mf_path, yeo1000_path], mask='MNI152_T1_2mm_brain_mask_dil')
display(correlations)

correlations = correlate_map_with_references(gsp1000mf_path, yeo1000_path, mask='MNI152_T1_2mm_brain_mask_dil')
correlations


array([0.35573652, 0.37528467], dtype=float32)

array([0.892824], dtype=float32)

**Get the difference map between GSP1000MF and Yeo1000 for Takotsubo Non-Medulla**

In [23]:
import os
import numpy as np
import pandas as pd
from nilearn import image
from nimlab.jax_functions import load_mask, load_image, NiftiMasker, correlate_map_with_references
from numpy.linalg import inv
from nilearn.image import resample_to_img, load_img

# Ensure we're in the correct directory
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')


gsp1000_ts_path = 'brain_maps/non_medulla_1000_mgh490_t_map.nii.gz'
yeo_ts_path = 'brain_maps/non_medulla_yeo_mgh490_t_map.nii.gz'

gsp1000_data = NiftiMasker().transform(gsp1000_ts_path)
yeo_data = NiftiMasker().transform(yeo_ts_path)

# Mask it to be positive
gsp1000_data = gsp1000_data.at[gsp1000_data < 0].set(0)
yeo_data = yeo_data.at[yeo_data < 0].set(0)

difference_data = gsp1000_data - yeo_data
# difference_data
difference_map = NiftiMasker().inverse_transform(difference_data)
difference_map.to_filename('brain_maps/ts_non-medulla_difference_map.nii.gz')